In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

import json
import matplotlib.pyplot as plt
import os
import seaborn as sns
import snowflake.connector
from skimage import io

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

print(session.connection.database)
print(session.connection.schema)
print(session.connection.role)

### Stops Data

**Description**: 

This table details the stops on given routes used by the transportation system.

**Notes**:
* There are ~8,100 stops across 87 routes.
* Many stops have identical locations, but correspond to different routes.
* Identical stops are distinguished by their "stop location ID".

In [ ]:
# Import data as a Pandas Dataframe and preview.
DATABASE_ROOT = 'TRANSPORTATION_HUB.HUB'

stops_data = session.sql(f"SELECT * FROM {DATABASE_ROOT}.STOPS").to_pandas()
stops_data

### Route Shapes Data

**Description**: 

This table contains the geographic data for each route.

**Notes**:
* There are 87 unique routes with shape data.
* However, routes often (but not always) have "Inbound" and "Outbound" variations.
* This results in a total of 181 route shapes.

In [ ]:
# Import data as a Pandas Dataframe and preview.
route_shape_data = session.sql(f"SELECT * FROM {DATABASE_ROOT}.ROUTE_SHAPES").to_pandas()
route_shape_data

In [ ]:
# Count the unique routes.
print(f"There are {route_shape_data['ROUTE_ID'].unique().size} unique routes.")

In [ ]:
# Filter down to an arbitrary route.
arbitrary_route_data = route_shape_data.loc[route_shape_data['ROUTE_SHAPE_ID'] == 1]
arbitrary_route_data

In [ ]:
# Extract that route's data and process into a data structure.
arbitrary_route_shape_data = json.loads(arbitrary_route_data['GEOGRAPHY'][0])
arbitrary_route_shape_data_coordinates = arbitrary_route_shape_data['coordinates']
long_vals = [x[0] for x in arbitrary_route_shape_data_coordinates]
lat_vals = [x[1] for x in arbitrary_route_shape_data_coordinates]

In [ ]:
# Draw the route.
plot = plt.plot(long_vals, lat_vals)
plt.title('1-Vermont: Inbound')
plot

In [ ]:
USE ROLE SYSADMIN;
CREATE OR REPLACE NETWORK RULE trimet_schedules_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('trimet.org');
USE ROLE ACCOUNTADMIN;
CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION trimet_schedules_access_integration
  ALLOWED_NETWORK_RULES = (trimet_schedules_rule)
  ENABLED = true;
GRANT USAGE ON INTEGRATION trimet_schedules_access_integration TO ROLE developer;
USE ROLE developer;

show integrations;

In [ ]:
# NOTE this does not work in snowflake notebook because the integration above is not i
# in scope???


# Check against the actual route map
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True

f = "https://trimet.org/schedules/img/001.png"
a = io.imread(f)

plt.imshow(a)
plt.axis('off')

plt.show()

In [ ]:
# Import data as a Pandas Dataframe and preview.
trip_delay_data = session.sql(f"SELECT * FROM {DATABASE_ROOT}.TRIP_DELAYS").to_pandas()
trip_delay_data

### Agencies (TBD)

### Routes (TBD)

### Stop Schedule (TBD)

### Stop Times (TBD)

### Trips (TBD)

### Vehicles (TBD)

### Vehicle Positions (TBD)